In [ ]:
from datasets import load_from_disk
from pathlib import Path



In [ ]:
DATA_ROOT = Path("./data")
PIXMO_SAVE_DIR = DATA_ROOT / "pixmo_cap_clean"
LIBRI_SAVE_DIR = DATA_ROOT / "librispeech_clean"



In [ ]:
pixmo_ds = DatasetDict.load_from_disk(str(PIXMO_SAVE_DIR))
pixmo_train = pixmo_ds["train"]
pixmo_val   = pixmo_ds["validation"]



In [ ]:
librispeech_ds = load_from_disk(str(LIBRI_SAVE_DIR))

print(len(pixmo_train), len(pixmo_val), len(librispeech_ds))


In [2]:
from imports.encoders import VisionEncoder, AudioEncoder
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vision_enc = VisionEncoder(
    model_name="facebook/dinov2-base",
    device=device,
    dtype=torch.float16,
)

audio_enc = AudioEncoder(
    model_name="openai/whisper-base",
    device=device,
    dtype=torch.float16,
)



In [ ]:
# Example: encode a batch of PIL images
from PIL import Image

imgs = [Image.open("/path/to/img1.jpg"), Image.open("/path/to/img2.jpg")]
vision_out = vision_enc.encode_images(imgs)
vision_feats, vision_mask = vision_out["feats"], vision_out["mask"]

# Example: encode a batch of audio waveforms (Tensor)
wav_batch = torch.randn(2, 16000 * 5)  # (B, T) fake data at 16kHz
audio_out = audio_enc.encode_waveforms(wav_batch, sample_rates=16000)
audio_feats, audio_mask = audio_out["feats"], audio_out["mask"]


In [ ]:
import torch
from Perciever import PerceiverLatentEncoder, ProjectorMLP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Run once to set feat_dim
encoder_out = vision_enc.encode_images(list_of_PIL_images)
feats, mask = encoder_out["feats"], encoder_out["mask"]   # (B, T, D_in), (B, T)

perceiver = PerceiverLatentEncoder(
    num_latents=64,
    d_latent=512,
    d_input=vision_enc.feat_dim,
    num_layers=4,
    num_heads=8,
    mlp_ratio=4.0,
    dropout=0.1,
).to(device)

proj_align = ProjectorMLP(
    d_in=512,
    d_out=1024,      # your alignment / MRL dim
    hidden_factor=2.0,
    dropout=0.1,
).to(device)

latents = perceiver(feats, encoder_mask=mask)     # (B, L, 512)
z = proj_align(latents)                           # (B, L, 1024)
z_pooled = z.mean(dim=1)                          # (B, 1024) if you want pooled
